In [2]:
import pandas as pd

import glob

from pathlib import Path
import csv

In [3]:
# Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json

import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [4]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [5]:
## Import symbol list 
path = Path('../Resources/short_list_for_ML.pkl')
key_list = load_obj(path)
len(key_list)

379

In [6]:
test_symbol = key_list[10]
symbol = 'ACWI'
symbol

'ACWI'

In [7]:
## Load Data
# df = pd.read_csv(
#     '../FilesExportIndividualStockDFs_Big/'+symbol+'_combined_df.csv')
# df
path = Path('../FilesExport_DFs_with_TI_shifted/'+symbol+'_df_with_shifted_TI.pkl')
data_import = load_obj(path)

df = data_import[symbol].copy()

## Set X and y data 
X = df.drop(columns={'close'}).values
y = df['close'].values

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

# load json and create model
file_path = Path('../Model_Data_shift_low_loss/'+symbol+'_model_data.json')
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

# load weights into new model
file_path = Path('../Model_Data_shift_low_loss/'+symbol+'_model_weights.h5')
loaded_model.load_weights(file_path)

## Compile loaded model and print mse score 
loaded_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])
score = loaded_model.evaluate(X, y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
print()

## take dataframe copy and compare predicted values to actual values 
df2 = df.copy()
df2['predicted'] = loaded_model.predict(X)
df_pred = df2[['close','predicted']]
df_pred

mse: 67.45%



,close,predicted
Date,,
2016-02-02,52.049999,51.913132
2016-02-03,52.610001,52.549870
2016-02-04,52.480000,52.510971
2016-02-05,52.200001,52.380890
2016-02-08,53.480000,53.558540
...,...,...
2021-10-04,105.270000,105.085083
2021-10-05,105.210000,105.178871
2021-10-06,104.690000,104.698059


In [8]:
score

[0.6744682788848877, 0.6744682788848877]

In [9]:
loaded_model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_39 (Dense)            (None, 8)                 168       
                                                                 
 dense_40 (Dense)            (None, 8)                 72        
                                                                 
 dense_41 (Dense)            (None, 1)                 9         
                                                                 
Total params: 249
Trainable params: 249
Non-trainable params: 0
_________________________________________________________________


In [10]:
df2 = df.copy()
#df2 = df2.tail(len(X_test_scaled))
df2['predicted'] = loaded_model.predict(X)
df_pred = df2[['close','predicted']]
df_pred

,close,predicted
Date,,
2016-02-02,52.049999,51.913132
2016-02-03,52.610001,52.549870
2016-02-04,52.480000,52.510971
2016-02-05,52.200001,52.380890
2016-02-08,53.480000,53.558540
...,...,...
2021-10-04,105.270000,105.085083
2021-10-05,105.210000,105.178871
2021-10-06,104.690000,104.698059


In [11]:
model_loss, model_accuracy = loaded_model.evaluate(X, y, verbose=0)
print(model_accuracy)

0.6744682788848877
